In [ ]:
%matplotlib qt
%load_ext autoreload   
%autoreload 2

In [ ]:
from vpm_py import VPM
from time import sleep
import numpy as np
from mpi4py import MPI
import numpy as np

from vpm_py.vpm_io import print_IMPORTANT, print_red, print_green, print_blue
from vpm_py.visualization import Particle3DPlot
from hill_spherical_vortex import hill_assign

In [ ]:
# Initialize MPI
comm = MPI.COMM_WORLD
start_time = MPI.Wtime()
rank = comm.Get_rank()
np_procs = comm.Get_size()

# Initialize VPM
vpm = VPM(
    max_particle_num= 10000,
    number_of_equations= 3,
    number_of_processors= np_procs,
    rank= rank,
    # d_particle_mesh= [0.2, 0.2, 0.2],
)

# PRINT THE RANK OF THE PROCESS AND DETERMINE HOW MANY PROCESSES ARE RUNNING
print_blue(f"Number of processes: {np_procs}", rank)
comm.Barrier()
print_blue(f"Rank: {rank}")
comm.Barrier()

In [ ]:
DT = 1 * 0.1
NI = -0.1
neq = 3 
UINF = np.array([0., 0., 0.])

# Create particles
NVR = 100
XPR_zero = np.zeros((3, NVR), dtype=np.float64)
QPR_zero = np.ones((neq + 1, NVR), dtype=np.float64)
UPR_zero = np.zeros((3, NVR), dtype=np.float64)
GPR_zero = np.zeros((3, NVR), dtype=np.float64)

In [ ]:
# Initialization VPM
comm.Barrier()
vpm.vpm(
    num_particles=NVR,
    num_equations=neq,
    mode = 0,
    particle_positions= XPR_zero, 
    particle_strengths= QPR_zero, 
    particle_velocities= UPR_zero, 
    particle_deformations= GPR_zero,
    timestep=0,
    viscosity=NI,
)
comm.Barrier()


In [ ]:
if rank == 0:
    st = MPI.Wtime()

print_IMPORTANT(f"Hill vortex initialization", rank)
_, RHS_pm_hill = hill_assign(
    Dpm= vpm.dpm,
    NN= vpm.nn,
    NN_bl= vpm.nn_bl,
    Xbound= vpm.xbound,
    neqpm= vpm.num_equations,
    a = 2.0,
    us = 1.0,
    z0 = 0.0,
)

vpm.set_rhs_pm(RHS_pm_hill)
print_red(f"Setting RHS_PM as computed from the hill vortex", rank)

if rank == 0:
    st = MPI.Wtime()
    print_red(f"Remeshing")
XPR, QPR, GPR, UPR = vpm.remesh_particles_3d(-1) 
if rank == 0:
    et = MPI.Wtime()
    print(f"\tRemeshing took {int((et - st) / 60)}m {int(et - st) % 60}s\n")

print_IMPORTANT(f"Particles initialized", rank)
vpm.particles.print_positions_fortran()

In [ ]:
# Create the plot to live update the particles
if rank == 0:
    plotter = Particle3DPlot()
    plotter.update(
        x = XPR[0,:],
        y = XPR[1,:],
        z = XPR[2,:],
        c = np.sqrt(QPR[0,:]**2 + QPR[1,:]**2 + QPR[2,:]**2)
    )

# ITERATION

In [ ]:
T = 0
max_iter = 100
def iteration(i,T):
    comm.Barrier()
    NVR = vpm.particles.NVR
    XPR = vpm.particles.XP
    QPR = vpm.particles.QP
    UPR = vpm.particles.UP
    GPR = vpm.particles.GP
    T += DT
    print_IMPORTANT(
        f"Iteration= {i} of {max_iter}\nT={T}\nDT={DT}",
        rank = rank,
        color_divider="green",
        color_text="green"
    )
    vpm.vpm(
        num_particles=NVR,
        num_equations=neq,
        mode = 2,
        particle_positions    =  XPR,
        particle_strengths    =  QPR,
        particle_velocities   =  UPR,
        particle_deformations =  GPR,
        timestep=i,
        viscosity=NI,
    )

    print_IMPORTANT(f"INFO", rank)
    if rank == 0:
        XPR = vpm.particles.XP
        QPR = vpm.particles.QP
        UPR = vpm.particles.UP
        GPR = vpm.particles.GP
        # Print the size of the particles

        print_IMPORTANT(f"Convecting Particles", rank)
        # # Move the particles
        for j in range(vpm.particles.NVR):
            XPR[:3, j] += (UPR[:3, j] + UINF) * DT
            FACDEF = 1.0
            QPR[:3, j] -= FACDEF * GPR[:3, j] * DT

        # Update the plot
        plotter.update(
            x = XPR[0,:],
            y = XPR[1,:],
            z = XPR[2,:],
            c = np.sqrt(QPR[0,:]**2 + QPR[1,:]**2 + QPR[2,:]**2)
        ) 

    sleep(0.5)
    print_IMPORTANT(f"Redefine Bounds", rank)

    comm.Barrier()
    vpm.vpm(
        num_particles=NVR,
        num_equations=neq,
        mode = 0,
        particle_positions    =  XPR,
        particle_strengths    =  QPR,
        particle_velocities   =  UPR,
        particle_deformations =  GPR,
        timestep=i,
        viscosity=NI,
    )
    comm.Barrier()

    if i%10 == 0:
        print_IMPORTANT(f"Remeshing", rank)
        # Remeshing
        if rank == 0:
            st = MPI.Wtime()
            print_red(f"Remeshing")
        XPR, QPR, UPR, GPR = vpm.remesh_particles_3d(1)
        if rank == 0:
            et = MPI.Wtime()
            print(f"\tRemeshing took {int((et - st) / 60)}m {int(et - st) % 60}s")

In [ ]:
for i in range(100):
    iteration(i, T)